<img src="https://raw.githubusercontent.com/Qiskit/qiskit-tutorials/master/images/qiskit-heading.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Repetition codes on real devices

* Requires qiskit-ignis 0.2.0 or greater.

### Contributors

James R. Wootton, IBM Research

In [another notebook](../../qiskit/ignis/repetition_code.ipynb) we saw how to use the `RepetitionCode` and `GraphDecoder` tools from Ignis. Here we'll specifcially look at how to use them with real hardware.

In [1]:
from qiskit.ignis.verification.topological_codes import RepetitionCode
from qiskit.ignis.verification.topological_codes import GraphDecoder
from qiskit.ignis.verification.topological_codes import lookuptable_decoding, postselection_decoding

Let's start off easy, with a `d=3`, `T=1` repetition code.

In [2]:
d = 3
T = 1
code = RepetitionCode(d,T)

The circuits for this are found in `code.circuit['0']` (for an encoded `0`) and code.circuit['1'] (for an encoded `1`). We can use `count_ops` to see which operations these contain.

In [3]:
code.circuit['0'].count_ops()

{'cx': 4, 'measure': 5, 'reset': 2, 'barrier': 1}

In [4]:
code.circuit['1'].count_ops()

{'x': 3, 'barrier': 2, 'cx': 4, 'measure': 5, 'reset': 2}

Both contain 4 `cx` gates, which is exactly what we'd expect from this code.

Now let's set up a real device as a backend, and compile these circuits for it.

In [7]:
from qiskit import IBMQ
from qiskit.compiler import transpile
from qiskit.transpiler import PassManager

provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_16_melbourne')
qc0 = transpile(code.circuit['0'], backend=backend)
qc1 = transpile(code.circuit['1'], backend=backend)

Let's see what has happened to the gates they contain.

In [8]:
print('gates for encoded 0 = ', qc0.count_ops())
print('gates for encoded 1 = ', qc1.count_ops())

gates for encoded 0 =  {'u2': 20, 'cx': 11, 'measure': 5, 'reset': 2, 'barrier': 1}
gates for encoded 1 =  {'u3': 3, 'barrier': 2, 'u2': 20, 'cx': 11, 'measure': 5, 'reset': 2}


The single qubit gates are now all `u2`s and `u3`s, as usually happens when we compile to real hardware. But the number of `cx` gates has increased! This implies some remapping has occurred, which is not what we want for error correction. To avoid this, we first need to provide look at the coupling map of the device.

In [9]:
coupling_map = backend.configuration().coupling_map
print(coupling_map)

[[1, 0], [1, 2], [2, 3], [4, 3], [4, 10], [5, 4], [5, 6], [5, 9], [6, 8], [7, 8], [9, 8], [9, 10], [11, 3], [11, 10], [11, 12], [12, 2], [13, 1], [13, 12]]


We can think of the qubits of a repetition code as sitting along a line (alternating between 'code' and 'link' qubits), with each qubit interacting only with its neighbours. So we need to look at the coupling map and find a line that covers as many qubits as we can. One possibility is

In [10]:
line = [0,1,2,3,4,5,6,8,9,10,11,12,13]

With this we can set up an `initial_layout` dictionary, which tells us exactly which qubit in the circuit corresponds to which qubits on the device.

In [14]:
def get_initial_layout(code,line):
    initial_layout = {}
    for j in range(d):
        initial_layout[code.code_qubit[j]] = line[2*j]
    for j in range(d-1):
        initial_layout[code.link_qubit[j]] = line[2*j+1]
    return initial_layout

initial_layout = get_initial_layout(code,line)

print(initial_layout)

{Qubit(QuantumRegister(3, 'code_qubit'), 0): 0, Qubit(QuantumRegister(3, 'code_qubit'), 1): 2, Qubit(QuantumRegister(3, 'code_qubit'), 2): 4, Qubit(QuantumRegister(2, 'link_qubit'), 0): 1, Qubit(QuantumRegister(2, 'link_qubit'), 1): 3}


Now let's compile using this.

In [15]:
qc0 = transpile(code.circuit['0'], backend=backend, initial_layout=initial_layout)
qc1 = transpile(code.circuit['1'], backend=backend, initial_layout=initial_layout)

print('gates for d = '+str(d)+' with encoded 0:', qc0.count_ops())
print('gates for d = '+str(d)+' with encoded 1:', qc1.count_ops())

gates for d = 3 with encoded 0: {'u2': 6, 'cx': 4, 'measure': 5, 'reset': 2, 'barrier': 1}
gates for d = 3 with encoded 1: {'u3': 3, 'barrier': 2, 'u2': 6, 'cx': 4, 'measure': 5, 'reset': 2}


The number of `cx` gates is now as it should be.

Before running, we need to remove the reset gates. For `T=1`, they don't actually do anything anyway. But since they aren't currently supported on hardware, they may cause use trouble.

In [16]:
def remove_reset(qc):
    qc.data = [ gate for gate in qc.data if gate[0].name!='reset' ]
    return qc
qc0 = remove_reset(qc0)
qc1 = remove_reset(qc1)

Now we can actually run the circuits. So let's set up a function to do this for us.

In [17]:
from qiskit import execute, Aer
from qiskit.providers.aer import noise

def get_syndrome(circuits,backend,sim=False,shots=8192):
    
    if sim:
        noise_model = noise.device.basic_device_noise_model(backend.properties())
        job = execute( circuits, Aer.get_backend('qasm_simulator'), noise_model=noise_model, shots=shots )
    else:
        job = execute( circuits, backend, shots=shots )
    raw_results = {}
    for log in ['0','1']:
        raw_results[log] = job.result().get_counts(log)

    return code.process_results( raw_results )

This has a `sim` argument, with which we can choose whether to actually use the real device, or just use the noise model we get from the device in a simulation.

Let's just simulate for now.

In [19]:
sim = True

In [20]:
results = get_syndrome([qc0,qc1],backend,sim=sim)
print(results)

{'0': {'1 0  01 11': 39, '1 1  00 11': 27, '0 1  01 00': 81, '0 0  01 10': 410, '1 1  01 10': 14, '0 1  00 10': 39, '1 0  00 01': 58, '1 0  11 01': 24, '1 1  01 01': 4, '0 0  10 01': 94, '1 1  10 10': 2, '1 1  00 00': 5, '0 1  10 11': 25, '1 0  10 11': 35, '1 0  00 10': 421, '0 0  10 10': 534, '0 0  11 00': 231, '0 1  01 11': 31, '0 1  00 01': 207, '1 1  11 11': 2, '0 0  01 01': 389, '1 0  01 00': 34, '0 1  11 01': 11, '1 1  10 01': 5, '0 1  10 00': 5, '0 0  11 11': 65, '0 1  11 10': 19, '0 0  00 11': 711, '1 0  10 00': 176, '0 0  00 00': 4468, '1 0  11 10': 26}, '1': {'1 0  01 11': 37, '0 0  01 10': 12, '1 1  01 10': 378, '0 1  00 10': 435, '0 0  10 01': 17, '1 1  01 01': 388, '0 1  10 11': 33, '1 0  00 10': 49, '0 0  10 10': 3, '0 1  01 11': 41, '0 1  00 01': 56, '0 0  11 11': 1, '1 1  11 00': 302, '1 0  11 10': 9, '1 1  00 11': 667, '0 1  01 00': 48, '1 0  00 01': 178, '1 0  11 01': 14, '1 1  10 10': 586, '1 1  00 00': 4291, '1 0  10 11': 37, '0 0  11 00': 3, '1 1  11 11': 80, '0 0 

And we can decode the results.

In [21]:
dec = GraphDecoder(code)

logical_prob_match = dec.get_logical_prob(results)
logical_prob_lookup = lookuptable_decoding(results,results)
logical_prob_post = postselection_decoding(results)

for log in ['0','1']:
    print('d =',d,',log =',log)
    print('logical error probability for matching      =',logical_prob_match[log])
    print('logical error probability for lookup table  =',logical_prob_lookup[log])
    print('logical error probability for postselection =',logical_prob_post[log])
    print('')

d = 3 ,log = 0
logical error probability for matching      = 0.0352783203125
logical error probability for lookup table  = 0.03857421875
logical error probability for postselection = 0.0011178180192264698

d = 3 ,log = 1
logical error probability for matching      = 0.0411376953125
logical error probability for lookup table  = 0.0361328125
logical error probability for postselection = 0.0006986492780624127



Now let's see what happens when we look at different code sizes.

In [22]:
for d in range(3,8):
    
    code = RepetitionCode(d,1)
    
    initial_layout = get_initial_layout(code,line)
    circuits = [ transpile(code.circuit[log], backend=backend, initial_layout=initial_layout) for log in ['0','1'] ]
    circuits = [ remove_reset(qc) for qc in circuits ]
    
    print('gates for d = '+str(d)+' with encoded 0:', circuits[0].count_ops(), '\n')

    results = get_syndrome(circuits,backend,sim=sim)   
    
    dec = GraphDecoder(code)

    logical_prob_match = dec.get_logical_prob(results)
    logical_prob_lookup = lookuptable_decoding(results,results)
    logical_prob_post = postselection_decoding(results)

    for log in ['0','1']:
        print('d =',d,',log =',log)
        print('logical error probability for matching      =',logical_prob_match[log])
        print('logical error probability for lookup table  =',logical_prob_lookup[log])
        print('logical error probability for postselection =',logical_prob_post[log])
        print('')
    print('')

gates for d = 3 with encoded 0: {'u2': 6, 'cx': 4, 'measure': 5, 'barrier': 1} 

d = 3 ,log = 0
logical error probability for matching      = 0.03466796875
logical error probability for lookup table  = 0.0379638671875
logical error probability for postselection = 0.0004439511653718091

d = 3 ,log = 1
logical error probability for matching      = 0.034423828125
logical error probability for lookup table  = 0.0301513671875
logical error probability for postselection = 0.0009170105456212746


gates for d = 4 with encoded 0: {'u2': 12, 'cx': 6, 'measure': 7, 'barrier': 1} 

d = 4 ,log = 0
logical error probability for matching      = 0.0247802734375
logical error probability for lookup table  = 0.0179443359375
logical error probability for postselection = 0.0

d = 4 ,log = 1
logical error probability for matching      = 0.020751953125
logical error probability for lookup table  = 0.015869140625
logical error probability for postselection = 0.0


gates for d = 5 with encoded 0: {'u2': 12, '

If we look only at odd `d`, here we see a steady decrease in the logical error probability, as expected. A decrease is not seen between each odd $d$ and the following $d+1$ for the matching decoder. This is due to the fact that the number of errors required to overturn a clear majority is the same in both cases.

Note that all the jobs above could have been sent in a single batch, which speeds things up for real devices. Also, two separate sets of results for each code should really be obtained and used in `lookuptable_decoding` to prevent over-fitting. These things were not done to keep this a simpler and clearer tutorial. 

In [23]:
keywords = {'Topics': ['Ignis', 'Quantum error correction'], 'Commands': ['`RepetitionCode`', '`GraphDecoder`', '`transpile`', '`initial_layout`']}